In [35]:

import pickle
import os
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "2"
os.system("nvcc --version")
import os
from glob import glob
import tensorflow as tf
import tensorflow_io as tfio
print(tf.__version__)
import numpy as np


2.10.1


In [34]:
import tensorflow as tf
import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.10.1
Keras version: 2.10.0


In [33]:
import keras
from keras import layers
from tensorflow.keras import ops

ImportError: cannot import name 'ops' from 'tensorflow.keras' (C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packages\keras\api\_v2\keras\__init__.py)

In [26]:
classNames = ['islet', 'epithelium', 'vasculature', 'fat', 'acini', 'stroma', 'nontissue', 'PanIN', 'nerves', 'immune', 'PDAC', 'black']
    
print(classNames)

['islet', 'epithelium', 'vasculature', 'fat', 'acini', 'stroma', 'nontissue', 'PanIN', 'nerves', 'immune', 'PDAC', 'black']


In [18]:
#_______________Create Tensorflow dataset______________

# Paths to training and validation datasets
pthDataset = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\02_23_2024'

# Define paths to training and validation directories
pthTrain = os.path.join(pthDataset, 'training')
pthValidation = os.path.join(pthDataset, 'validation')

# Get paths to training images and labels
train_images = sorted(glob(os.path.join(pthTrain, 'im', "*.tif")))
train_masks = sorted(glob(os.path.join(pthTrain, 'label', 'corrected_labels', "*.tif")))

# Get paths to validation images and labels
val_images = sorted(glob(os.path.join(pthValidation, 'im', "*.tif")))
val_masks = sorted(glob(os.path.join(pthValidation, 'label', 'corrected_labels', "*.tif")))

# Define constants
IMAGE_SIZE = 1024
BATCH_SIZE = 4
NUM_TRAIN_IMAGES = len(train_images)  # Number of training images
NUM_VAL_IMAGES = len(val_images)  # Number of validation images

NUM_CLASSES = len(classNames)  # Number of classes


def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tfio.experimental.image.decode_tiff(image)
        image = tf.image.rgb_to_grayscale(image)  # Convert to grayscale if it's not already
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tfio.experimental.image.decode_tiff(image)
        if tf.shape(image)[-1] == 1:  # If grayscale, convert to RGB
            image = tf.image.grayscale_to_rgb(image)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    return tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]

def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 1024, 1024, 1), dtype=tf.float32, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 1024, 1024, 1), dtype=tf.float32, name=None))>


In [36]:
import tensorflow as tf
import os
from glob import glob
import cv2

# Paths to training and validation datasets
pthDataset = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\02_23_2024'

# Define paths to training and validation directories
pthTrain = os.path.join(pthDataset, 'training')
pthValidation = os.path.join(pthDataset, 'validation')

# Get paths to training images and labels
train_images = sorted(glob(os.path.join(pthTrain, 'im', "*.tif")))
train_masks = sorted(glob(os.path.join(pthTrain, 'label', 'corrected_labels', "*.tif")))

# Get paths to validation images and labels
val_images = sorted(glob(os.path.join(pthValidation, 'im', "*.tif")))
val_masks = sorted(glob(os.path.join(pthValidation, 'label', 'corrected_labels', "*.tif")))

# Define constants
IMAGE_SIZE = 1024
BATCH_SIZE = 4
NUM_TRAIN_IMAGES = len(train_images)
NUM_VAL_IMAGES = len(val_images)

NUM_CLASSES = len(classNames)  # Assuming classNames is defined elsewhere

def read_tif(image_path, mask=False):
    def parse_with_opencv(path):
        img = cv2.imread(path.decode('UTF-8'), cv2.IMREAD_UNCHANGED)
        if mask:
            if len(img.shape) == 3:
                img = img[:,:,0]  # Take first channel if multi-channel
            img = img[:,:,np.newaxis]  # Add channel dimension
        else:
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
        return img.astype(np.float32)
    
    image = tf.numpy_function(parse_with_opencv, [image_path], tf.float32)
    if mask:
        image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    else:
        image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    return image

def load_data(image_path, mask_path):
    image = read_tif(image_path)
    mask = read_tif(mask_path, mask=True)
    return image, mask

def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 1024, 1024, 1), dtype=tf.float32, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 1024, 1024, 1), dtype=tf.float32, name=None))>


In [23]:
##_____________________Building the DeepLabV3+ model_______________________________
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return ops.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
        interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output


def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

AttributeError: module 'keras.applications' has no attribute 'resnet50'